In [30]:
from webcolors import name_to_rgb
import plotly.io as pio


In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import plotly_express as px
import plotly.graph_objs as go
# https://blog.dominodatalab.com/fitting-gaussian-process-models-python/
def exponential_cov(x, y, params):
    return params[0] * np.exp( -0.5 * params[1] * np.subtract.outer(x, y)**2)

def conditional(x_new, x, y, params):
 
    B = exponential_cov(x_new, x, params)
    C = exponential_cov(x, x, params)
    A = exponential_cov(x_new, x_new, params)

    mu = np.linalg.inv(C).dot(B.T).T.dot(y)
    sigma = A - B.dot(np.linalg.inv(C).dot(B.T))

    return(mu.squeeze(), sigma.squeeze())

In [ ]:
import matplotlib.pylab as plt
 
θ = [1, 10]
σ_0 = exponential_cov(0, 0, θ)
xpts = np.arange(-3, 3, step=0.01)
plt.errorbar(xpts, np.zeros(len(xpts)), yerr=σ_0, capsize=0)


In [ ]:
x = [1.]
y = [np.random.normal(scale=σ_0)]
σ_1 = exponential_cov(x, x, θ)
 
def predict(x, data, kernel, params, sigma, t):
    k = [kernel(x, y, params) for y in data]
    Sinv = np.linalg.inv(sigma)
    y_pred = np.dot(k, Sinv).dot(t)
    sigma_new = kernel(x, x, params) - np.dot(k, Sinv).dot(k)
    return y_pred, sigma_new
 
x_pred = np.linspace(-3, 3, 1000)
predictions = [predict(i, x, exponential_cov, θ, σ_1, y) for i in x_pred]

y_pred, sigmas = np.transpose(predictions)
plt.errorbar(x_pred, y_pred, yerr=sigmas, capsize=0)
plt.plot(x, y, "ro")

In [ ]:
m, s = conditional([-0.7], x, y, θ)
y2 = np.random.normal(m, s)

In [ ]:
x.append(-0.7)
y.append(y2)
 
σ_2 = exponential_cov(x, x, θ)
predictions = [predict(i, x, exponential_cov, θ, σ_2, y) for i in x_pred]

y_pred, sigmas = np.transpose(predictions)
plt.errorbar(x_pred, y_pred, yerr=sigmas, capsize=0)
plt.plot(x, y, "ro")

In [ ]:
x_more = [-2.1, -1.5, 0.3, 1.8, 2.5]
mu, s = conditional(x_more, x, y, θ)
y_more = np.random.multivariate_normal(mu, s)
y_more

In [ ]:
x += x_more
y += y_more.tolist()
 
σ_new = exponential_cov(x, x, θ)
predictions = [predict(i, x, exponential_cov, θ, σ_new, y) for i in x_pred]
 
y_pred, sigmas = np.transpose(predictions)
plt.errorbar(x_pred, y_pred, yerr=sigmas, capsize=0)
plt.plot(x, y, "ro")

In [ ]:
df = pd.DataFrame(dict(x_pred=x_pred, y_pred=y_pred, sigmas=sigmas))

In [ ]:
trace_mean = go.Scatter(
    x=df.x_pred,
    y=df.y_pred,
    mode='lines',
    line=dict(color='rgb(231,107,243)'),
    name='mean',
)

In [ ]:
trace_upper = go.Scatter(
    x=df.x_pred,
    y=df.y_pred+sigmas,
    mode='lines',
    line=dict(color='rgba(231,107,243,0.25)'),
    name='upper bound',
    showlegend=False,
)

In [ ]:
trace_lower = go.Scatter(
    x=df.x_pred,
    y=df.y_pred-sigmas,
    mode='lines',
    line=dict(color='rgba(231,107,243,0.25)'),
    name='lower bound',
    fill='tonexty',
    fillcolor='rgba(231,107,243,0.2)',
    showlegend=False,
)

In [22]:
fig = go.FigureWidget(
    data=[trace_mean,trace_upper,trace_lower],
    layout=dict(
        height=800, 
        xaxis=dict(title='x'), 
        yaxis=dict(title='pred y'))
)
fig

FigureWidget({
    'data': [{'line': {'color': 'rgb(231,107,243)'},
              'mode': 'lines',
           …

In [23]:
fig.layout.font.size = 16

In [33]:
continuous_error_bars = dict(
    data = dict(
        scatter=[
            dict(
                name='mean',
                mode='lines',
                line=dict(
                    color='rgb(231, 107, 243)',
                ),
            ),
            dict(
                mode='lines',
                name='upper bound',
                showlegend=False,
                line=dict(
                    color='rgba(231,107,243,0.25)',
                ),
            ),
            dict(
                mode='lines',
                line=dict(color='rgba(231,107,243,0.25)'),
                name='lower bound',
                fill='tonexty',
                fillcolor='rgba(231,107,243,0.2)',
                showlegend=False,
            )
        ]
    ),
    layout= dict(
        height=800, 
        xaxis=dict(title='x'), 
        yaxis=dict(title='pred y')
    )
)
pio.templates['continuous_error_bars'] = continuous_error_bars

In [40]:
fig = go.FigureWidget(
    data=[
        dict(
            x=df.x_pred,
            y=df.y_pred,
        ),
        dict(
            x=df.x_pred,
            y=df.y_pred+sigmas),
        dict(    
            x=df.x_pred,
            y=df.y_pred-sigmas)
    ],
)
fig.layout.template = 'plotly_dark+continuous_error_bars+presentation'
fig

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '58082e4e-ae62-4bc5-af87-d0b09e170118',
 …

In [58]:
pink = name_to_rgb('pink')
pink_rgb_str = 'rgb({r}, {g}, {b})'.format(r=pink.red,g=pink.green,b=pink.blue)
pink_rgba_str = 'rgba({r}, {g}, {b}, {a})'.format(r=pink.red,g=pink.green,b=pink.blue, a=0.25)
pink_fill_rgba_str = 'rgba({r}, {g}, {b}, {a})'.format(r=pink.red,g=pink.green,b=pink.blue, a=0.2)
pink_lines = dict(
    data = dict(
        scatter=[
            dict(
                line=dict(
                    color=pink_rgb_str,
                ),
            ),
            dict(
                line=dict(
                    color=pink_rgba_str,
                ),
            ),
            dict(
                line=dict(color=pink_rgba_str),
                fillcolor=pink_fill_rgba_str,
            )
        ]
    ),
)
pio.templates['pink_lines'] = pink_lines

In [93]:
def line_color_template_factory(color):
    pink = name_to_rgb(color)
    pink_rgb_str = 'rgb({r}, {g}, {b})'.format(r=pink.red,g=pink.green,b=pink.blue)
    pink_rgba_str = 'rgba({r}, {g}, {b}, {a})'.format(r=pink.red,g=pink.green,b=pink.blue, a=0.25)
    pink_fill_rgba_str = 'rgba({r}, {g}, {b}, {a})'.format(r=pink.red,g=pink.green,b=pink.blue, a=0.2)
    pink_lines = dict(
        data = dict(
            scatter=[
                dict(
                    line=dict(
                        color=pink_rgb_str,
                    ),
                ),
                dict(
                    line=dict(
                        color=pink_rgba_str,
                    ),
                ),
                dict(
                    line=dict(color=pink_rgba_str),
                    fillcolor=pink_fill_rgba_str,
                )
            ]
        ),
    )
    #pio.templates['{}_lines'.format(color)] = pink_lines
    #return '{}_lines'.format(color)
    return pink_lines

In [129]:
continuous_error_bars = dict(
    data = dict(
        scatter=[
            dict(
                name='mean',
                mode='lines',
            ),
            dict(
                mode='lines',
                name='upper bound',
                showlegend=False,
            ),
            dict(
                mode='lines',
                name='lower bound',
                fill='tonexty',
                showlegend=False,
            )
        ]
    ),
    layout= dict(
        height=800, 
        xaxis=dict(title='x'), 
        yaxis=dict(title='pred y'),
        hovermode='x',
    )
)
pio.templates['continuous_error_bars'] = continuous_error_bars

In [200]:
def line(df, x, y, error_y, color='blue', template='plotly_white',**layout):
    line_color_template = line_color_template_factory(color)
    fig = go.FigureWidget(
        data=[
            dict(
                x=df[x],
                y=df[y],
            ),
            dict(
                x=df[x],
                y=df[y]+df[error_y]),
            dict(
                x=df[x],
                y=df[y]-df[error_y])
        ],
        layout=layout,
    )
    fig.layout.template.update(line_color_template)
    for t in template.split('+'):
        fig.layout.template.update(pio.templates[t])
    fig.layout.template.update(continuous_error_bars)
    return fig

In [229]:
def linec(df, fig, x, y, error_y, color='blue', template='plotly_white',**layout):
    line_color_template = line_color_template_factory(color)
#     fig = go.FigureWidget(
    data=(
        dict(
            type='scatter',
            x=df[x],
            y=df[y],
        ),
        dict(
            type='scatter',
            x=df[x],
            y=df[y]+df[error_y]),
        dict(
            type='scatter',
            x=df[x],
            y=df[y]-df[error_y])
    )
#         layout=layout,
#     )
    fig = go.FigureWidget(data=fig.data+data, layout=fig.layout)
    fig.layout.template.update(line_color_template)
    for t in template.split('+'):
        fig.layout.template.update(pio.templates[t])
    fig.layout.template.update(continuous_error_bars)
    return fig

In [231]:
fig = go.FigureWidget()
fig = df.pipe(linec, fig, x='x_pred', y='y_pred', error_y='sigmas', color='limegreen', template='ggplot2+presentation', width=1500, height=800)
fig

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '3f598a48-c06b-4037-bf9d-d0211cd2b949',
 …

In [201]:
fig = df.pipe(line, x='x_pred', y='y_pred', error_y='sigmas', color='limegreen', template='ggplot2+presentation', width=1500, height=800)
fig

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'c09d3a4d-b781-4da4-bf31-4fc873c7f8d0',
 …

In [92]:
continuous_error_bars.update(pink_lines)
continuous_error_bars

{'data': {'scatter': [{'line': {'color': 'rgb(255, 192, 203)'}},
   {'line': {'color': 'rgba(255, 192, 203, 0.25)'}},
   {'line': {'color': 'rgba(255, 192, 203, 0.25)'},
    'fillcolor': 'rgba(255, 192, 203, 0.2)'}]},
 'layout': {'height': 800,
  'xaxis': {'title': 'x'},
  'yaxis': {'title': 'pred y'}}}

In [108]:
continuous_error_bars

{'data': {'scatter': [{'name': 'mean', 'mode': 'lines'},
   {'mode': 'lines', 'name': 'upper bound', 'showlegend': False},
   {'mode': 'lines',
    'name': 'lower bound',
    'fill': 'tonexty',
    'showlegend': False}]},
 'layout': {'height': 800,
  'xaxis': {'title': 'x'},
  'yaxis': {'title': 'pred y'}}}

In [86]:
fig.layout.template.update(pio.templates['plotly_dark'])

layout.Template({
    'data': {'bar': [{'marker': {'line': {'color': 'rgb(17,17,17)', 'width': 0.5}}, 'type': 'bar'}],
             'barpolar': [{'marker': {'line': {'color': 'rgb(17,17,17)', 'width': 0.5}}, 'type': 'barpolar'}],
             'carpet': [{'aaxis': {'endlinecolor': '#A2B1C6',
                                   'gridcolor': '#506784',
                                   'linecolor': '#506784',
                                   'minorgridcolor': '#506784',
                                   'startlinecolor': '#A2B1C6'},
                         'baxis': {'endlinecolor': '#A2B1C6',
                                   'gridcolor': '#506784',
                                   'linecolor': '#506784',
                                   'minorgridcolor': '#506784',
                                   'startlinecolor': '#A2B1C6'},
                         'type': 'carpet'}],
             'choropleth': [{'colorbar': {'outlinewidth': 0, 'ticks': ''}, 'type': 'choropleth'}],
        

In [246]:
def background_images():
    return dict(layout=dict(
        images=[
            dict(
                name='background',
                xref= "x",
                yref= "y",
                x= 0,
                y= 0,
                layer= "below",
                sizing="stretch",
            ),
            dict(
                name='watermark',
                xref='paper',
                yref='paper',
                x=.75,
                y=0.3,
                sizex=0.25,
                sizey=0.25,
                sizing="stretch",
                opacity=0.2,
                layer="below"
            )
        ]
    ))

images_template = dict(layout=dict(
    images=[
        dict(
            name='background',
            xref= "x",
            yref= "y",
            x= 0,
            y= 0,
            layer= "below",
            sizing="stretch",
        ),
        dict(
            name='watermark',
            xref='paper',
            yref='paper',
            x=.75,
            y=0.3,
            sizex=0.25,
            sizey=0.25,
            sizing="stretch",
            opacity=1,
            layer="below"
        )
    ]
))
pio.templates['images'] = images_template

In [237]:
import numpy as np
from PIL import Image

image_array = np.random.randint(0, 255, size=(100, 100)).astype('uint8')
image = Image.fromarray(image_array)


trace1= go.Scatter(x=[0,0.5,1,2,2.2],y=[1.23,2.5,0.42,3,1])
fig=go.FigureWidget(
    data=[trace1],
    layout=dict(
        images=[dict(
            source="https://raw.githubusercontent.com/michaelbabyn/plot_data/master/naphthalene.png",
            sizex=100,
            sizey=100,
            templateitemname='background',
        )],
        xaxis=dict(range=[0,150]),
        yaxis=dict(range=[150,0]),
        width=800,
        #height=800,
        template=background_images(),
    )
)
#fig.layout.template.update(background_images())
fig

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '10b00881-d5e1-4d85-b776-711cba6a29fe',
 …

In [ ]:
import scipy.misc
scipy.misc.imresize

In [162]:
pio.templates['plotly'].layout
fig.layout.template.layout.images

(layout.Image({
     'layer': 'below', 'sizing': 'stretch', 'x': 0, 'xref': 'x', 'y': 0, 'yref': 'y'
 }),)

In [257]:
fig = df.pipe(
    line, 
    x='x_pred', 
    y='y_pred', 
    error_y='sigmas', 
    color='limegreen', 
    template='images+ggplot2+presentation', 
    width=800, 
    height=800,
    images=[dict(
        source="https://raw.githubusercontent.com/michaelbabyn/plot_data/master/naphthalene.png",
        templateitemname='watermark',
    )],
)
fig

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'd481c755-ec60-4f4c-be62-d259dcf2fce8',
 …

In [258]:
fig.add_scatter(x=x, y=y, mode='markers')

Scatter({
    'mode': 'markers',
    'uid': '1b7c29b2-1ce7-4b59-8099-3c9844cc5054',
    'x': [1.0, -0.7, -2.1, -1.5, 0.3, 1.8, 2.5],
    'y': [-2.760696905467817, 1.1547042973875987, 0.2498761114900528,
          -1.961592228048777, 0.2026166241590379, -0.39499233588427407,
          1.319410273877994]
})

In [516]:
import inspect

out = tuple(filter(lambda x: x[0].startswith('add'), inspect.getmembers(go.FigureWidget)))

In [789]:

def maybe_col(data, name):
    return data[name] if name in data.columns else name

@pd.api.extensions.register_dataframe_accessor("plot")
class PlotlyAccessor(object):
    def __init__(self, pandas_obj):
        self._obj = pandas_obj
        self.init()
        intercept_methods = tuple(filter(lambda x: x[0].startswith('add'), inspect.getmembers(go.FigureWidget)))
        [setattr(self, name, self.resolves_kwargs(value)) for name, value in intercept_methods]
        
    
    def _resolve_data(self, name):
        return self._obj[name] if name in self._obj.columns else name
        
    def _is_valid_method(self, name):
        return False if name.startswith('_') else True
    
    def show(self):
        return self._figure
        
    def query(self, q):
        self._obj = self._obj.query(q)
        return self
        
        
    def _groups_plotter(self, fn, *args, **kwargs):
        for name, group in self._groups:
            kw = dict([k, maybe_col(group, v) if isinstance(v,str) else v] for k,v in kwargs.items())
            fn(self._figure, *args, name=name, **kw)
        return self
        
    def resolves_kwargs(self, fn):
        def wrapper(*args, **kwargs):
            if self._groups is not None:
                return self._groups_plotter(fn, *args, **kwargs)
            fn(self._figure, *args, **dict([k, self._resolve_data(v) if isinstance(v,str) else v] for k,v in kwargs.items()))
            return self
        return wrapper

    def layout(self, **kwargs):
        self._figure.layout.update(kwargs)
        return self
    
    def add_template(self, template):
        self._figure.layout.template.update(template)
        return self
    
    def init(self):
        self._figure = go.FigureWidget(layout=dict(template=images_template))
        self._groups = None
        return self
    
    def clear(self):
        self._figure.update(data=[],layout={})
        
        return self
    
    def groupby(self, *args, **kwargs):
        self._groups = self._obj.groupby(*args,**kwargs)
        return self
    
    def add_image(self, template='background', **kwargs):
        kwargs['templateitemname'] = template
        self._figure.layout.images = self._figure.layout.images + (kwargs,)
        return self
        

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning:

registration of accessor <class '__main__.PlotlyAccessor'> under name 'plot' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.



In [738]:
df = pd.DataFrame(dict(x_pred=x_pred, y_pred=y_pred, sigmas=sigmas))
df2 = df.assign(ub=lambda df: df.y_pred+df.sigmas, lb=lambda df: df.y_pred-df.sigmas)

ValueError: 
    Invalid element(s) received for the 'images' property of layout
        Invalid elements include: [1]

    The 'images' property is a tuple of instances of
    Image that may be specified as:
      - A list or tuple of instances of plotly.graph_objs.layout.Image
      - A list or tuple of dicts of string/value properties that
        will be passed to the Image constructor

        Supported dict properties:
            
            layer
                Specifies whether images are drawn below or
                above traces. When `xref` and `yref` are both
                set to `paper`, image is drawn below the entire
                plot area.
            name
                When used in a template, named items are
                created in the output figure in addition to any
                items the figure already has in this array. You
                can modify these items in the output figure by
                making your own item with `templateitemname`
 

In [757]:
plotter = (df2.plot
 .add_template(dict(layout=pio.templates['plotly_dark'].layout))
 .add_template(continuous_error_bars)
 .add_scatter(x='x_pred', y='y_pred', )
 .add_scatter(x='x_pred', y='ub')
 .add_scatter(x='x_pred', y='lb')
)


In [769]:

plotter = (df2.plot
 .add_template(dict(layout=pio.templates['plotly_dark'].layout))
 .add_template(line_color_template_factory('limegreen'))
)


In [759]:
df2.plot.show()

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '4c192328-2c98-4b9a-950e-db07e290e916',
 …

In [753]:
plotter.add_scatter(x=x,y=y,mode='markers').add_scatter(x=x,y=y,mode='markers')


In [731]:
df2.plot.query('x_pred<0').add_scatter(x='x_pred', y='y_pred', )

In [732]:
df2.plot

In [790]:


import plotly_express as px
gapminder = px.data.gapminder()
gapminder2007 = gapminder.query("year == 2007")

px.scatter(gapminder2007, x="gdpPercap", y="lifeExp")



In [617]:
go.FigureWidget(
    data=[
        go.Scatter(
            x=gapminder2007['gdpPercap'],
            y=gapminder2007['lifeExp'],
            marker=dict( 
                symbol='circle',
                size=gapminder2007['pop'].values, 
                sizemode='area', 
                sizeref=366300.85999999999),
            mode='markers'
        )
    ]
)

FigureWidget({
    'data': [{'marker': {'size': array([  31889923,    3600523,   33333216,   12420476,   40301…

In [677]:
t = {'data': {'scatter': [{'mode': 'markers'}]}}

In [793]:
gapminder2007.plot.init()
gapminder2007.plot.groupby('continent')
(gapminder2007.plot
 .add_scatter(x='gdpPercap', y='lifeExp')
 .add_template(t)
 .add_image(
     source="https://raw.githubusercontent.com/michaelbabyn/plot_data/master/naphthalene.png",
     template='watermark')
)

In [794]:
gapminder2007.plot.show()

FigureWidget({
    'data': [{'name': 'Africa',
              'type': 'scatter',
              'uid': '43a4977b…

In [600]:
a = gapminder2007['pop']-gapminder2007['pop'].min()
b = a/a.max()*(60-5)+5


In [598]:
b.min()

10.0

In [566]:
foo = dict(marker=dict(size=10, foo='bar'), name='foo')
foo

{'marker': {'size': 10, 'foo': 'bar'}, 'name': 'foo'}

In [570]:
foo.update(marker=dict(size=11))
foo

{'marker': {'size': 11}, 'name': 'bar'}